In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn import metrics
import glob
import os

In [2]:
# Read in all merged csv data files
path = r'../data/curated/merged_dataset/' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

merged_df = pd.concat(li, axis=0, ignore_index=True)
merged_df.drop(['address', 'latitude', 'longitude', 'postcode', 'sa2_2016'], axis=1, inplace=True)

print(merged_df.columns)
#pd.get_dummies(merged_df['sa2_2021'])
merged_df.rename({'gdp(USD Millioins)': 'gdp', 'saving_rate(% of GDP)': 'saving_rate'}, axis=1, inplace=True)
merged_df = pd.get_dummies(data=merged_df, columns=['sa2_2021'], prefix='sa2')
merged_df = pd.get_dummies(data=merged_df, columns=['residence_type'], prefix='resiType') 
#merged_df.drop(['sa2_2021', 'residence_type'], axis=1, inplace=True)
merged_df.dropna(inplace=True)

y = merged_df['weekly_rent']
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(merged_df.isna().sum())
merged_df    

Index(['year', 'sa2_2021', 'residence_type', 'nbed', 'nbath', 'ncar',
       'min_distance_to_cbd', 'min_distance_to_park', 'min_distance_to_prim',
       'min_distance_to_second', 'min_distance_to_train',
       'min_distance_to_hosp', 'min_distance_to_poli', 'min_distance_to_shop',
       'weekly_rent', 'gdp(USD Millioins)', 'saving_rate(% of GDP)',
       'income_per_person', 'population_density', 'crime_cases'],
      dtype='object')


,year,nbed,nbath,ncar,min_distance_to_cbd,min_distance_to_park,min_distance_to_prim,min_distance_to_second,min_distance_to_train,min_distance_to_hosp,...,sa2_217031473,sa2_217031474,sa2_217031475,sa2_217031476,sa2_217041477,sa2_217041478,sa2_217041479,sa2_217041480,resiType_Apartment,resiType_House
0,2013,2.0,1.0,0,227.97163,23.16035,7.35747,16.96507,35.56825,21.35025,...,0,0,0,0,0,0,0,0,0,1
1,2013,2.0,1.0,0,223.66084,5.71742,6.50536,6.76794,7.54355,7.42972,...,0,0,0,0,0,0,0,0,0,1
2,2013,2.0,1.0,0,243.25680,5.11222,0.20027,36.72106,50.85341,36.63541,...,0,0,0,0,0,0,0,0,0,1
3,2013,4.0,2.0,0,140.35827,78.32509,10.66523,11.91899,11.26906,177.44731,...,0,0,0,0,0,0,0,0,0,1
4,2013,1.0,1.0,0,13.86135,0.93250,1.32931,3.49174,2.20800,177.44731,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172030,2022,3.0,1.0,1,293.28053,0.56012,1.21809,114.77016,90.08591,140.56888,...,0,0,0,0,0,0,0,0,0,1
172031,2022,3.0,2.0,2,258.29111,3.49087,5.08707,3.60570,8.37185,2.60312,...,0,0,0,0,0,0,1,0,0,1
172032,2022,2.0,2.0,1,9.47077,2.45011,1.33931,1.62322,3.63291,140.56888,...,0,0,0,0,0,0,0,0,0,1
172033,2022,1.0,1.0,1,1.84933,0.65199,1.10438,1.27940,1.87840,140.56888,...,0,0,0,0,0,0,0,0,1,0


In [7]:
len(merged_df.loc[merged_df['year'] == 2022])

53696

In [3]:
all_candidates = ['nbed', 'nbath', 'ncar', 'min_distance_to_cbd', 'min_distance_to_park',
       'min_distance_to_prim', 'min_distance_to_second',
       'min_distance_to_train', 'min_distance_to_hosp', 'min_distance_to_poli',
       'min_distance_to_shop', 'gdp', 'saving_rate', 'income_per_person',
       'population_density', 'crime_cases']
all_X = merged_df[['year']+list(merged_df.filter(regex='sa2'))+list(merged_df.filter(regex='resiType'))] # null predictors
all_X

,year,sa2_201011001,sa2_201011002,sa2_201011005,sa2_201011006,sa2_201011007,sa2_201011008,sa2_201011481,sa2_201011482,sa2_201011483,...,sa2_217031473,sa2_217031474,sa2_217031475,sa2_217031476,sa2_217041477,sa2_217041478,sa2_217041479,sa2_217041480,resiType_Apartment,resiType_House
0,2013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172030,2022,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
172031,2022,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
172032,2022,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
172033,2022,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
model = XGBRegressor()
model.fit(all_X, y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [ ]:
# holdout method to split data into train and test
num = (df_taxi_gas.count())*0.8
df_tr = df_taxi_gas.loc[:num[0], :]
df_te = df_taxi_gas.loc[num[0]:, :]

X_train = df_tr.drop(columns='avg_income_rate')
Y_train = df_tr['avg_income_rate']

y_test = df_te["avg_income_rate"]
x_test = df_te.drop(columns="avg_income_rate")

In [ ]:
# build XGBoost regression model
# https://machinelearningmastery.com/xgboost-for-regression/
model = XGBRegressor()
model.fit(X_train, Y_train)